
!pip install mmcv-full==1.3.18 -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.7.0/index.html
#install mmcv  mmcv的版本要修改
!rm  -rf  mmsegmentation
!git  clone  https://github.com/open-mmlab/mmsegmentation.git 
%cd  mmsegmentation
!pip install -e .

In [1]:
import torch
from torch import nn
import  torch.nn.functional as F
from torchvision import models

ModuleNotFoundError: No module named 'torch'

In [ ]:
#数据的参数文件,随便找一个文件参数
from mmcv import Config
cfg = Config.fromfile('configs/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes.py')
#print(f'Config:\n{cfg.pretty_text}')

In [ ]:
#仔细修改各个部分的参数

# Since we use ony one GPU, BN is used instead of SyncBN 模型参数，保证模型中的BN层都是一样的
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg


cfg.model.pop('auxiliary_head')
cfg.model.backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0,1,2,3 ),
        strides= (1,2,2,2,),
        style='pytorch')
cfg.model.pretrained =None
#根据自己模型需要的参数给定，但是要注意继承基类需要的参数也要给定，不然还是修改其内部函数
cfg.model.decode_head = dict(type='GCNHead',
                             out_channels=21,
                             kernel_size=3,
                             input_channels = [256,512,1024,2048],
                            in_index=3,
                            channels=256,
                            num_classes=21,
                            norm_cfg=cfg.norm_cfg,
                            align_corners=False,
                            loss_decode=dict(
                            type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0))

#注意模型的一下参数到底是要放在什么位置上的
cfg.train_cfg = cfg.model.train_cfg
cfg.test_cfg =cfg.model.test_cfg
cfg.model.pop('train_cfg')
cfg.model.pop('test_cfg')

#GPU
cfg.gpu_ids = range(1)

In [ ]:
class GCN(nn.Module):
    def __init__(self,in_channels,
                        out_channels,
                        kernel_size):
        super(GCN,self).__init__()
        self.conv1_1=nn.Conv2d(in_channels,out_channels,(kernel_size,1),padding=1)
        self.conv1_2 = nn.Conv2d(out_channels,out_channels,(1,kernel_size))
    
        self.conv2_1 = nn.Conv2d(in_channels,out_channels,(1,kernel_size),padding=1)
        self.conv2_2 = nn.Conv2d(out_channels,out_channels,(kernel_size,1))

        self.init_weights()
    def forward(self,x):
        out = self.conv1_2(self.conv1_1(x))
        out += self.conv2_2(self.conv2_1(x))
        return out
    def init_weights(self):
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                nn.init.kaiming_normal(m.weight)
                
class BRModule(nn.Module):
    
    def __init__(self,in_channes,
                out_channels,
                ):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channes,out_channels,3,padding=1)
        self.relu =nn.ReLU(True)
        self.conv2 = nn.Conv2d(in_channes,out_channels,3,padding=1)
        self.init_weights()
    def forward(self,x):
        residual = x
        x =self.conv1(x)
        x= self.relu(x)
        x= self.conv2(x)
        x +=residual
        return x
    def init_weights(self):
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)

class GCNLayer(nn.Module):
    
    def __init__(self,in_channels,out_channels,kernel_size):
        super().__init__()
        self.gcn = GCN(in_channels,out_channels,kernel_size)
        self.bn=BRModule(out_channels,out_channels)
    def forward(self,x):
        x =self.gcn(x)
        x =self.bn(x)
        return x
    


In [ ]:
from mmseg.models.builder import HEADS
from mmseg.models.decode_heads.decode_head import BaseDecodeHead

#修改需要使用这个重新注册
HEADS.module_dict.pop('GCNHead')
@HEADS.register_module()
class GCNHead(BaseDecodeHead):
    
    def __init__(self,input_channels,out_channels,kernel_size,**kwargs):
        super(GCNHead,self).__init__(input_channels,**kwargs)#参数传给基类
        
        #这里要注意，一定要把模块使用这种形式，不然无法注册到模型中去
        #或者使用列表[]，再用nn.Sequential(**list)的形式，不过这个是针对序列模型
        self.gcn_layer = nn.ModuleList()
        self.deconv_layer =nn.ModuleList()
        
        for i in range(len(input_channels)):
            self.gcn_layer.append(GCNLayer(input_channels[i],out_channels,kernel_size))
            self.deconv_layer.append(nn.ConvTranspose2d(out_channels,out_channels,2,2))
            
        #这里也是注册到模型中
        self.br_layer = nn.ModuleList()
        for _ in range(len(input_channels)-1):
            self.br_layer.append(BRModule(out_channels,out_channels))
        self.br1 = BRModule(out_channels,out_channels)
        self.br2 = BRModule(out_channels,out_channels)
        self.deconv =nn.ConvTranspose2d(out_channels,out_channels,2,2)
    def forward(self,out):
        x = self.deconv_layer[-1](self.gcn_layer[-1](out[-1]))
        
        for i in reversed(range(len(out)-1)):
            x +=self.gcn_layer[i](out[i])
            
            x =self.br_layer[i](x)
            x =self.deconv_layer[i](x)
        x =self.br1(x)
        x =self.deconv(x)
        x =self.br2(x)
        return x
    def _init_inputs(self, in_channels, in_index, input_transform=None):
        pass
    
    def extra_repr(self):
        s = f"GCNHEAD"
        return s


In [ ]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor
from mmseg.models.backbones.resnet import ResNet
from mmseg.models.builder import BACKBONES
# Build the detector
model = build_segmentor(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

In [ ]:

#测试模型
device =torch.device('cuda:0')
test =torch.randn(1,3,512,512).to(device)
print(test.device)
model.eval()
model.cuda()

In [ ]:
forward = model.forward
model.forward = model.forward_dummy

In [ ]:
#计算每一层的FLOPS
from mmcv.cnn.utils import get_model_complexity_info
#查看函数支持计算的层有哪些
#model.forward = model.forward_dummy
#需要对HEADs里实现 extra_repr,随便输出什么字符串即可
get_model_complexity_info(model,(3,512,512))

# 数据集参数

In [ ]:
# Modify dataset type and path数据集的参数的预先设定
cfg.dataset_type = 'StandfordBackgroundDataset'
cfg.data_root = data_root
cfg.train_cfg = dict()
cfg.test_cfg = dict(mode='whole')

#batch大小
cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu=8
#标准化参数，要计算的
cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

cfg.crop_size = (256, 256)
#数据处理方式
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(320, 240), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(320, 240),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]

#数据集，需要加入label路径
#类名
cfg.data.train.type = cfg.dataset_type
#参数
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_dir = img_dir
cfg.data.train.ann_dir = ann_dir
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = '/kaggle/working/mmsegmentation/mmsegmentation/splits/train.txt' 

#类名
cfg.data.val.type = cfg.dataset_type
#参数
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_dir = img_dir
cfg.data.val.ann_dir = ann_dir
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = '/kaggle/working/mmsegmentation/mmsegmentation/splits/val.txt'

#类名
cfg.data.test.type = cfg.dataset_type
#参数
cfg.data.test.data_root = cfg.data_root
cfg.data.test.img_dir = img_dir
cfg.data.test.ann_dir = ann_dir
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = 'splits/val.txt'
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch模型参数下载
#cfg.load_from = 'checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'

# Set up working dir to save files and logs.保存路径
cfg.work_dir = './work_dirs/tutorial'
#这个是配合val
cfg.evaluation = dict(interval=1, metric='accuracy')

# 训练设置

In [ ]:
cfg.log_config = dict(interval=50, hooks=[dict(type='TextLoggerHook', by_epoch=False)])
cfg.dist_params = dict(backend='nccl')
cfg.log_level = 'INFO'
cfg.load_from = None
cfg.resume_from = None


cfg.workflow = [('train', 1)]
cfg.cudnn_benchmark = True

#momentum，weight_decay参数设置
cfg.optimizer = dict(type='SGD', lr=0.004, momentum=0.9, weight_decay=0.0001)
cfg.optimizer_config = dict()
#学习率变化
cfg.lr_config = dict(policy='poly', power=0.9, min_lr=0.0001, by_epoch=False)
#训练器
cfg.runner = dict(type='IterBasedRunner', max_iters=40000)
#保存模型
cfg.checkpoint_config = dict(by_epoch=False, interval=4000)
#评估方法
cfg.evaluation = dict(interval=4000, metric='mIoU', pre_eval=True)
#继续训练
cfg.load_from = 'checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'

# Set up working dir to save files and logs.保存路径
cfg.work_dir = './work_dirs/tutorial'

# 训练

In [ ]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor


# Build the dataset，
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)
# Add an attribute for visualization convenience 这里就是用到训练集的标签信息
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=False, 
                meta=dict())
